## AgglomerativeClustering Record Linkage pt2

In [101]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from scipy.spatial.distance import euclidean
import pandas as pd
from itertools import combinations

In [127]:
import warnings

# Ignora tutti i warning
warnings.filterwarnings("ignore")

# Codice che genera warning

# Riabilita i warning
warnings.filterwarnings("default")


## Test semplice

In [40]:
X = np.array([[1, 2], [1, 4], [1, 0],[4, 2],[4, 4],[4, 0],[1,2]])
clustering = AgglomerativeClustering(None,distance_threshold=1).fit(X)

In [41]:
clustering

AgglomerativeClustering(distance_threshold=1, n_clusters=None)

In [42]:
clustering.labels_

array([0, 4, 3, 5, 2, 1, 0])

## Scelta della threshold

In [76]:
def compute_threshold(elements):
    pairs = list(combinations(elements,2))
    # Calcola la distanza euclidea tra ciascuna coppia di vettori
    distances = [euclidean(pair[0], pair[1]) for pair in pairs]

    # Calcola la media delle distanze
    mean_distance = np.mean(distances)

    print("Media distanza tra i vettori di embedding:", mean_distance)

    return mean_distance

## Caricamento dello schema mediato con cluster (Strategia Embedding Word2Vec)

In [106]:
schema_mediato = pd.read_csv("SchemaMediatoConCluster.csv")

/Users/alessandropesare/anaconda3/envs/FinalProjectIDD3_8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: DtypeWarning: Columns (4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [107]:
schema_mediato
schema_mediato.drop('Unnamed: 0',axis = 1,inplace=True)

In [109]:
schema_mediato

,name,location,marketCap,sharePrice,category,foundationYear,block
0,Lacework,"United States, San Jose",$8.3B,NaN,Cybersecurity,2015,110
1,Tipalti,"United States, San Mateo",$8.3B,NaN,Fintech,2010,198
2,Tempus,"United States, Chicago",$8.1B,NaN,Health,2015,1504
3,Anduril,"United States, Irvine",$8.48B,NaN,Artificial intelligence,2017,480
4,Bolt,"Estonia, Tallinn",$8.4B,NaN,Auto & transportation,2013,659
...,...,...,...,...,...,...,...
81701,PepsiCo,"Purchase, NY, United States",$241.05 Billion,NaN,NaN,NaN,1066
81702,Costco,"Issaquah, WA, United States",$258.15 Billion,NaN,NaN,NaN,0
81703,Broadcom,"San Jose, CA, United States",$265.27 Billion,NaN,NaN,NaN,1018
81704,Coca-Cola,"Atlanta, GA, United States",$273.20 Billion,NaN,NaN,NaN,1843


In [108]:
schema_mediato.drop('cluster',axis = 1,inplace=True)

In [49]:
num_clusters = schema_mediato['cluster'].nunique()
print(num_clusters)

2167


In [110]:
bert_vectors = pd.read_csv("MediatedSchemaVectors.csv")

## Clustering con vettori di embedding estratti tramite BERT

In [116]:
stats = {}

In [136]:
for i in range(0,num_clusters):
    print("Cluster corrente: ",i)
    records_cluster = schema_mediato[schema_mediato['block']==i]
    if(len(records_cluster)>1):
        vectors_cluster = bert_vectors.iloc[records_cluster.index] #lista degli indici di riga per il cluster corrente
        distance_threshold = compute_threshold(vectors_cluster.values.tolist())
        model = AgglomerativeClustering(n_clusters=None,distance_threshold=0.25*distance_threshold).fit(vectors_cluster)
        stats[i] = {}
        vectors_cluster['cluster'] = model.labels_
        for j in range(0,max(model.labels_)):
            stats[i][j] = vectors_cluster[vectors_cluster['cluster']==j].index
    else:
        stats[i] = {}
        stats[i][0] = records_cluster.index


Cluster corrente:  0
Media distanza tra i vettori di embedding: 0.01632243717808641
Cluster corrente:  1


/var/folders/ss/s59pk8px01vb8p_b48z9wxz40000gn/T/ipykernel_34966/3382395784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vectors_cluster['cluster'] = model.labels_


Media distanza tra i vettori di embedding: 0.014988829223350692
Cluster corrente:  2


/var/folders/ss/s59pk8px01vb8p_b48z9wxz40000gn/T/ipykernel_34966/3382395784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vectors_cluster['cluster'] = model.labels_


Media distanza tra i vettori di embedding: 0.008301759274622442
Cluster corrente:  3


...

/var/folders/ss/s59pk8px01vb8p_b48z9wxz40000gn/T/ipykernel_34966/3382395784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vectors_cluster['cluster'] = model.labels_
/var/folders/ss/s59pk8px01vb8p_b48z9wxz40000gn/T/ipykernel_34966/3382395784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vectors_cluster['cluster'] = model.labels_
/var/folders/ss/s59pk8px01vb8p_b48z9wxz40000gn/T/ipykernel_34966/3382395784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

Cluster corrente:  2123
Media distanza tra i vettori di embedding: 0.012544305976549323
Cluster corrente:  2124
Media distanza tra i vettori di embedding: 0.008391560669121594
Cluster corrente:  2125
Media distanza tra i vettori di embedding: 0.016106827698884427
Cluster corrente:  2126
Media distanza tra i vettori di embedding: 0.012123814365621526
Cluster corrente:  2127
Media distanza tra i vettori di embedding: 0.016314845268420926
Cluster corrente:  2128
Media distanza tra i vettori di embedding: 0.006078964715759347
Cluster corrente:  2129
Media distanza tra i vettori di embedding: 0.011912072877297895
Cluster corrente:  2130
Media distanza tra i vettori di embedding: 0.01112983660409415
Cluster corrente:  2131
Media distanza tra i vettori di embedding: 0.012416783836698687
Cluster corrente:  2132
Media distanza tra i vettori di embedding: 0.013679191889434795
Cluster corrente:  2133
Media distanza tra i vettori di embedding: 0.014502789352925611
Cluster corrente:  2134
Media dis

/var/folders/ss/s59pk8px01vb8p_b48z9wxz40000gn/T/ipykernel_34966/3382395784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vectors_cluster['cluster'] = model.labels_
/var/folders/ss/s59pk8px01vb8p_b48z9wxz40000gn/T/ipykernel_34966/3382395784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vectors_cluster['cluster'] = model.labels_
/var/folders/ss/s59pk8px01vb8p_b48z9wxz40000gn/T/ipykernel_34966/3382395784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

Media distanza tra i vettori di embedding: 0.01512076637757568
Cluster corrente:  2136
Media distanza tra i vettori di embedding: 0.00871184481329633
Cluster corrente:  2137
Media distanza tra i vettori di embedding: 0.011977366286368326
Cluster corrente:  2138
Media distanza tra i vettori di embedding: 0.013054156491555509
Cluster corrente:  2139
Media distanza tra i vettori di embedding: 0.009047839673355186
Cluster corrente:  2140
Media distanza tra i vettori di embedding: 0.008234740382647371
Cluster corrente:  2141
Media distanza tra i vettori di embedding: 0.016965200508013797
Cluster corrente:  2142
Media distanza tra i vettori di embedding: 0.020026862552572043
Cluster corrente:  2143
Media distanza tra i vettori di embedding: 0.01242213190848461
Cluster corrente:  2144
Media distanza tra i vettori di embedding: 0.014708521147585479
Cluster corrente:  2145
Media distanza tra i vettori di embedding: 0.012723088173730761
Cluster corrente:  2146
Media distanza tra i vettori di emb

/var/folders/ss/s59pk8px01vb8p_b48z9wxz40000gn/T/ipykernel_34966/3382395784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vectors_cluster['cluster'] = model.labels_
/var/folders/ss/s59pk8px01vb8p_b48z9wxz40000gn/T/ipykernel_34966/3382395784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vectors_cluster['cluster'] = model.labels_
/var/folders/ss/s59pk8px01vb8p_b48z9wxz40000gn/T/ipykernel_34966/3382395784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [155]:
schema_mediato.iloc[18575]

name              Simhapuri Energy
location          Hyderabad, India
marketCap                      NaN
sharePrice                     NaN
category                     Power
foundationYear               2005 
block                         2151
Name: 18575, dtype: object

In [156]:
schema_mediato.iloc[13832]

name                          R.K.M Powergen
location          Chennai, Tamil Nadu, India
marketCap                                NaN
sharePrice                               NaN
category                               Power
foundationYear                         2004 
block                                   2151
Name: 13832, dtype: object

In [144]:
schema_mediato.iloc[18717]

name                 Pharma Companies
location                          NaN
marketCap                         NaN
sharePrice                        NaN
category          Healthcare / Pharma
foundationYear                    NaN
block                               0
Name: 18717, dtype: object

In [145]:
schema_mediato.iloc[18700]

name                 Pharma Companies
location                          NaN
marketCap                         NaN
sharePrice                        NaN
category          Healthcare / Pharma
foundationYear                    NaN
block                               0
Name: 18700, dtype: object

In [149]:
stats

{0: {0: Int64Index([68317, 68468], dtype='int64'),
  1: Int64Index([66585, 66684], dtype='int64'),
  2: Int64Index([18700, 18717], dtype='int64'),
  3: Int64Index([78878, 80254], dtype='int64'),
  4: Int64Index([66238, 69829], dtype='int64'),
  5: Int64Index([81185], dtype='int64'),
  6: Int64Index([81567], dtype='int64'),
  7: Int64Index([81705], dtype='int64'),
  8: Int64Index([81702], dtype='int64'),
  9: Int64Index([19627, 25883], dtype='int64'),
  10: Int64Index([81531], dtype='int64'),
  11: Int64Index([80832], dtype='int64'),
  12: Int64Index([80574], dtype='int64'),
  13: Int64Index([81543], dtype='int64'),
  14: Int64Index([81548], dtype='int64'),
  15: Int64Index([81535], dtype='int64'),
  16: Int64Index([81522], dtype='int64'),
  17: Int64Index([81394], dtype='int64'),
  18: Int64Index([81070], dtype='int64'),
  19: Int64Index([81560], dtype='int64'),
  20: Int64Index([80795], dtype='int64'),
  21: Int64Index([81056], dtype='int64'),
  22: Int64Index([80857], dtype='int64'),

In [147]:
elementi_con_dimensione_maggiore_di_1 = sum(len(d) > 1 for d in stats.values())
elementi_con_dimensione_maggiore_di_1

2029

## Salvataggio su File

In [150]:
with open("./stats_clustersW2V.txt", "w") as file:
    # Iterare sul dizionario e scrivere ogni coppia chiave-valore su una nuova riga
    for chiave, valore in stats.items():
        file.write(f"{chiave}: {valore}\n")

## Caricamento dello schema mediato con cluster (Rule Based)

In [62]:
schema_mediato_2 = pd.read_csv("MediatedSchemaSemicolonLabel.csv",sep=';')

In [63]:
schema_mediato_2

,name,location,marketCap,sharePrice,category,foundationYear,Label
0,GRUPO TELEVISA,Mexico,NaN,NaN,"Communication Services, Media, Advertising, Br...",1973,1
1,WARNER MUSIC GROUP CORP.,USA,NaN,NaN,"Communication Services, Entertainment, Music",1958,2
2,PIRAMAL ENTERPRISES,India,NaN,NaN,"Healthcare, Conglomerate, Real Estate, Pharmac...",1984,3
3,Snam S.p.A.,Italy,$19.27 B,NaN,"Utilities, Utilities—Regulated Gas",NaN,4
4,Wheels Emi,"Pune,Maharashtra, India",NaN,NaN,NBFC,2017,5
...,...,...,...,...,...,...,...
81701,Semrush,USA,$1.12 B,$7.95,"Software, Tech, Internet",NaN,323
81702,Capricor Therapeutics,NaN,$0.11 B,$4.38,NaN,NaN,47
81703,Atacadão,Brazil,5810000000.0,3.0,NaN,NaN,476
81704,DV8 Public Company Ltd,"Bangkok, Thailand",฿1.04 Billion,NaN,NaN,NaN,69


In [65]:
num_clusters_2 = schema_mediato_2['Label'].nunique()
print(num_clusters_2)

7504


## Clustering con vettori di embedding estratti tramite BERT

In [ ]:
stats_2 = {}

In [ ]:
for i in range(0,num_clusters_2):
    print("Cluster corrente: ",i)
    records_cluster = schema_mediato[schema_mediato_2['label']==i]
    if(len(records_cluster)>1):
        vectors_cluster = bert_vectors.iloc[records_cluster.index] #lista degli indici di riga per il cluster corrente
        distance_threshold = compute_threshold(vectors_cluster.values.tolist())
        model = AgglomerativeClustering(n_clusters=None,distance_threshold=0.33*distance_threshold).fit(vectors_cluster)
        stats_2[i] = {}
        vectors_cluster['cluster'] = model.labels_
        for j in range(0,max(model.labels_)):
            stats_2[i][j] = vectors_cluster[vectors_cluster['cluster']==j].index
    else:
        stats_2[i] = {}
        stats_2[i][0] = records_cluster.index


## Salvataggio su File

In [ ]:
with open("./stats_clustersW2V.txt", "w") as file:
    # Iterare sul dizionario e scrivere ogni coppia chiave-valore su una nuova riga
    for chiave, valore in stats.items():
        file.write(f"{chiave}: {valore}\n")